# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных

In [1]:
# Импорт библиотеки

import pandas as pd


In [2]:
try:
    credit = pd.read_csv('/datasets/data.csv')
except:
    credit = pd.read_csv('data.csv')
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Открыли файл и методом info() посмотрели данные датафрейма. Всего имеем 21525 строк.

Проанализировав типы серий данных, можно заметить, days_employed - общий трудовой стаж в днях имеет тип float64. Скорее всего это не тот тип, так как стаж в днях это целое число. Возможно это не будет создавать проблемы в будущем, так как столбец не участвует в вопросах исследования.

Посмотрим состав уникальных значений в каждом столбце.

In [3]:
credit['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В колонке количество детей есть отрицательное число -1 и максимальное количество детей 20, т.е.данные скорее всего содержат ошибки.

In [4]:
credit['days_employed'].value_counts()

-8437.673028      1
-3507.818775      1
 354500.415854    1
-769.717438       1
-3963.590317      1
                 ..
-1099.957609      1
-209.984794       1
 398099.392433    1
-1271.038880      1
-1984.507589      1
Name: days_employed, Length: 19351, dtype: int64

В колонке days_employed (общий трудовой стаж в днях) видим отрицательные дробные числа. И присутствут пропуски, так как строк должно быть 21525, а их 19351.

In [5]:
credit['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В колонке dob_years (возвраст клиентв в годах) видим неявный пропуск: у 101 заемщика возраст равен 0. 

In [6]:
credit['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Столбец education (уровень образования клиента) содержит одни и те же категории данных в разном регистре.

In [7]:
credit['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

В данном столбце family_status (семейное положение) визуальных ошибок нет.

In [8]:
credit['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В столбце gender (пол) содержится 1 запись XNA.  

In [9]:
credit['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

В столбце income_type (тип занятости) ошибок нет. 

In [10]:
credit['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

В стоблце debt (имел ли задолженность по возврату кредитов) есть все данные.

In [11]:
credit['total_income'].value_counts()

253875.639453    1
157691.851056    1
70113.902786     1
116196.518662    1
157205.834562    1
                ..
168880.592469    1
148042.721049    1
60039.334460     1
175979.762960    1
82047.418899     1
Name: total_income, Length: 19351, dtype: int64

В столбце total_income (ежемесячный доход) есть пропущенные данные NaN, хотя они нужны в данном исследовании.

In [12]:
credit['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Столбец purpose (цель получения кредита) содержит одни и те же категории, сформулированные разным языком, что непосредственно в таком виде мешает категоризации.

### Шаг 2.1 Заполнение пропусков

Проверяем если ли пропущенные значения медотом isna().

In [13]:
credit.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [14]:
'пропущенные значения составляют {:.0%}'.format(credit['days_employed'].isna().sum()/len(credit['children']))

'пропущенные значения составляют 10%'

Пропущенные значения есть в двух стобцах: days_employed и total_income. И количество этих пропущенных значений одинаково. И составляет около 10% от всех данных в этих стоблцах. Это значительный процент и мы не можем просто удалить эти строки. Будем приводить их к нужному виду.

В столбце days_employed указывается общий трудовой стаж в днях, а в столбце total_income - ежемесячный доход. Предполагаю, что данные в этих столбцах отсутствует по следующим возможным причинам: 

- эти данные выгружаются из одной программы и произошел какой-то сбой
- клиенты не указали эти данные, тогда не понятно как им вообще выдали кредит

Заполним эти пропуски.

Ранее мы обнаружили, что в столбце 'days_employed' есть отрицательные значения. Утраним эту ошибку. Кроме того, для удобства переведем стаж в годы.

In [15]:
credit['days_employed'] = credit['days_employed'].abs()
credit['days_employed'] = credit['days_employed']/365
credit['days_employed']

0         23.116912
1         11.026860
2         15.406637
3         11.300677
4        932.235814
            ...    
21520     12.409087
21521    942.294258
21522      5.789991
21523      8.527347
21524      5.437007
Name: days_employed, Length: 21525, dtype: float64

Теперь увидели новую ошибку: есть стаж более 900 лет))) Посмотрим у кого такой стаж. Отфильтруем всех у кого стаж оказался более 55 лет. А так же посмотрим у кого менее 55 лет.

In [16]:
immortal = credit.loc[credit['days_employed']>55]

immortal['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

In [17]:
normal = credit.loc[credit['days_employed']<=55]

normal['income_type'].value_counts()

сотрудник          10014
компаньон           4577
госслужащий         1312
студент                1
предприниматель        1
в декрете              1
Name: income_type, dtype: int64

Получается нереальный стаж у пенсионеров и безработных. Предполагаю, что была допущенная ошибка в этих двух категориях. И стаж указывался не днях, а в часах. Поэтому разделим эти значения на 24часа.
Для проверки найдем максимальное значение в столбце days_employed.

In [18]:
credit.loc[credit['income_type']=='пенсионер','days_employed'] = credit['days_employed']/24
credit.loc[credit['income_type']=='безработный','days_employed'] = credit['days_employed']/24
print('Максимальный стаж ',credit['days_employed'].max())
credit['days_employed']

Максимальный стаж  50.38068465909146


0        23.116912
1        11.026860
2        15.406637
3        11.300677
4        38.843159
           ...    
21520    12.409087
21521    39.262261
21522     5.789991
21523     8.527347
21524     5.437007
Name: days_employed, Length: 21525, dtype: float64

Теперь можно заменить пропущенные значения в этом столбце на среднее значение, сгруппировав по типу деятельности. 

И снова проверим количество пропусков.

In [19]:
group_days_employed = credit.groupby('income_type')['days_employed'].mean()
for i in group_days_employed.index:
    credit.loc[(credit['income_type'] == i) & (credit['days_employed'].isna()), 'days_employed'] = group_days_employed[i]

credit.isna().sum()
#credit['days_employed']

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Теперь заполним пропуски в столбце total_income (ежемесячный доход). 
Найдем максимальное и минимальное значения в этом столбце

In [20]:
print ('min=',credit['total_income'].min())
print ('max=',credit['total_income'].max())

min= 20667.26379327158
max= 2265604.028722744


Разрыв между минимальным и максимальным значениями очень большой, поэтому будем заполнять пропуски медианным значением. 

Кроме того сделаем группировку по типу занятости.

In [21]:
group_total_income = credit.groupby('income_type')['total_income'].median()
for i in group_total_income.index:
    credit.loc[(credit['income_type'] == i) & (credit['total_income'].isna()), 'total_income'] = group_total_income[i]

In [22]:
display(credit.groupby('income_type')['total_income'].median().sort_values()) # выведем список медианных значений по категорям для наглядности
display(credit.loc[[12,26,29,41,55,65,67,82,83,90], ['total_income', 'income_type']]) # выведем 10 строк к которых были пропуски в столбце 'total_income' 


income_type
в декрете           53829.130729
студент             98201.625314
пенсионер          118514.486412
безработный        131339.751676
сотрудник          142594.396847
госслужащий        150447.935283
компаньон          172357.950966
предприниматель    499163.144947
Name: total_income, dtype: float64

,total_income,income_type
12,118514.486412,пенсионер
26,150447.935283,госслужащий
29,118514.486412,пенсионер
41,150447.935283,госслужащий
55,118514.486412,пенсионер
65,172357.950966,компаньон
67,118514.486412,пенсионер
82,142594.396847,сотрудник
83,142594.396847,сотрудник
90,142594.396847,сотрудник


Снова проверим данные на пропущенные значения

In [23]:
credit.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.

Проверим данные на аномалии, дубликаты.

Начнем со столбца children. Ранее мы обнаружили, что там есть значения -1 и 20. Скорее всего были допущены ошибки. Исправим их: -1 заменим на 1, 20 заменим на 2.

In [24]:
credit.loc[credit['children'] == -1, 'children'] = 1
credit.loc[credit['children'] == 20, 'children'] = 2
credit['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Остальные уникальные значения все допустимы.

Перейдем к следующему столбцу dob_years - возраст клиента в годах. Ранее обнаружили, что у некоторых клиентов стоит возраст 0. Заполним возвраст медианным значение, сгруппировав по типу занятости.

Проверим уникальные значения в столбце, чтобы убедится что нет 0.

In [25]:
group_dob_years = credit.groupby('income_type')['dob_years'].median()
for i in group_dob_years.index:
    credit.loc[(credit['income_type'] == i) & (credit['dob_years']==0), 'dob_years'] = group_dob_years[i]

    
#print(group_dob_years)
credit['dob_years'].value_counts()

39.0    648
35.0    617
40.0    615
41.0    607
34.0    603
38.0    598
42.0    597
33.0    581
31.0    560
36.0    555
44.0    547
29.0    545
30.0    540
48.0    538
37.0    537
50.0    514
43.0    513
32.0    510
49.0    508
28.0    503
45.0    497
27.0    493
56.0    487
52.0    484
47.0    480
54.0    479
46.0    475
58.0    461
57.0    460
53.0    459
51.0    448
59.0    444
55.0    443
26.0    408
60.0    397
25.0    357
61.0    355
62.0    352
63.0    269
64.0    265
24.0    264
23.0    254
65.0    194
22.0    183
66.0    183
67.0    167
21.0    111
68.0     99
69.0     85
70.0     65
71.0     58
20.0     51
72.0     33
19.0     14
73.0      8
74.0      6
75.0      1
Name: dob_years, dtype: int64

### Шаг 2.3. Изменение типов данных.

Мы определяли проблему в типе даннных days_employed. Так как это количество дней, то число должно быть целым.
Так же заменим вещественный тип данных на целочисленный у столбца total_income. Копейки в доходе - несущественная информация для исследования.


In [26]:
credit['days_employed'] = credit['days_employed'].astype('int')
credit['total_income'] = credit['total_income'].astype('int')

credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Ранее мы обнаружили, что в столбце education (уровень образования клиента) содержатся одни и те же категории данных в разном регистре. Исправим это. 

In [27]:
credit['education'] = credit['education'].str.lower()
credit['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Проверим на дупликаты.

In [28]:
credit.duplicated().sum()

71

Удалим все дубликаты.

In [29]:
credit = credit.drop_duplicates().reset_index()

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два словаря: education и family. Сохраним в каждом id и наименовнание значения.

In [30]:
education = credit[['education_id','education']]
education.head(10)

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее
5,0,высшее
6,0,высшее
7,1,среднее
8,0,высшее
9,1,среднее


In [31]:
family = credit[['family_status_id','family_status']]
family.head(10)

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак
5,1,гражданский брак
6,0,женат / замужем
7,0,женат / замужем
8,1,гражданский брак
9,0,женат / замужем


Найдем в новых датафреймах дубликаты и удалим их.

In [32]:
education.duplicated().sum()

21449

In [33]:
education = education.drop_duplicates().reset_index()
education

,index,education_id,education
0,0,0,высшее
1,1,1,среднее
2,13,2,неоконченное высшее
3,31,3,начальное
4,2962,4,ученая степень


In [34]:
family.duplicated().sum()

21449

In [35]:
family = family.drop_duplicates().reset_index()
family

,index,family_status_id,family_status
0,0,0,женат / замужем
1,4,1,гражданский брак
2,18,2,вдовец / вдова
3,19,3,в разводе
4,24,4,Не женат / не замужем


Получили два новых "словаря", в которых расшифровывается какой id чему соответствует. Теперь можно удалить из основной таблицы столбец education и family_status, тем самым ее оптимизируем.

In [36]:

credit = credit.drop(['family_status','education'], axis=1)

credit.head(10)

,index,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,23,42.0,0,0,F,сотрудник,0,253875,покупка жилья
1,1,1,11,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,2,0,15,33.0,1,0,M,сотрудник,0,145885,покупка жилья
3,3,3,11,32.0,1,0,M,сотрудник,0,267628,дополнительное образование
4,4,0,38,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,5,0,2,27.0,0,1,M,компаньон,0,255763,покупка жилья
6,6,0,7,43.0,0,0,F,компаньон,0,240525,операции с жильем
7,7,0,0,50.0,1,0,M,сотрудник,0,135823,образование
8,8,2,18,35.0,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,9,0,5,41.0,1,0,M,сотрудник,0,144425,покупка жилья для семьи


### Шаг 2.6. Категоризация дохода.

Категоризируем доход в соответствии со следующей градацией:
- 0-30000 -'E'
- 30001-50000 - 'D'
- 50001-200000 - 'C'
- 200001-1000000 - 'B'
- 1000001 и выше - 'A'

И добавим новый столбец с этой категорией total_income_category.

Напишем функцию, которая будет проверять сумму на попадание в нужные диапазоны.

In [37]:
def category(row):
    if row <= 30000:
        return 'E'
    if row >= 30001 and row <= 50000:
        return 'D'
    if row >= 50001 and row <= 200000:
        return 'C'
    if row >= 200001 and row <= 1000000:
        return 'B'
    if row >=1000001:
        return 'A'

#category(3000000)

credit['total_income_category'] = credit['total_income'].apply(category)
credit.head()

,index,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,0,1,23,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,1,11,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,2,0,15,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,3,11,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,4,0,38,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


In [38]:
credit['total_income_category'].value_counts()

C    16015
B     5042
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### Шаг 2.7. Категоризация целей кредита.

Столбец purpose (цель получения кредита) содержит одни и те же категории, сформулированные разным языком, что непосредственно в таком виде мешает категоризации. 
Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:
- операции с автомобилем
- операции с недвижимостью
- проведение свадеб
- получение образования

In [39]:
def purpose_category(row):
    if 'автомобил'in row:
        return 'операции с автомобилем'
    if 'недвижим'in row or 'жиль' in row or 'строител' in row or 'ремонт' in row:
        return 'операции с недвижимостью'
    if 'свадьб'in row:
        return 'проведение свадеб'
    if 'образов'in row:
        return 'получение образования'

#purpose_category ('отремонтировать квартиру')

credit['purpose_category'] = credit['purpose'].apply(purpose_category)
display(credit.head(10))
credit['purpose_category'].value_counts()

,index,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,0,1,23,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,1,11,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,2,0,15,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,3,11,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,4,0,38,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадеб
5,5,0,2,27.0,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,6,0,7,43.0,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,7,0,0,50.0,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,8,2,18,35.0,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадеб
9,9,0,5,41.0,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадеб            2324
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?


In [40]:
debt_children = pd.DataFrame()
debt_children['debt_children_count'] = credit.groupby('children')['debt'].count()
debt_children['debt_children_sum'] = credit.groupby('children')['debt'].sum()
debt_children['debt_children_ratio'] = debt_children['debt_children_sum']/debt_children['debt_children_count']*100
debt_children = debt_children.sort_values(by='debt_children_ratio', ascending=False)

debt_children

,debt_children_count,debt_children_sum,debt_children_ratio
children,,,
4,41,4,9.756098
2,2128,202,9.492481
1,4855,445,9.165808
3,330,27,8.181818
0,14091,1063,7.543822
5,9,0,0.000000


##### Вывод 1:

Получается, что наличие детей отрицательно влияет на возврат кредита в срок. Как можно заметить из таблицы невозврат кредита в срок у бездетных заемщиков 7,5%, а у заемщиков с детьми вырастает до 9,7%.

##### Вопрос 2:
Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [41]:
debt_family_status = pd.DataFrame()
debt_family_status['debt_family_status_count'] = credit.groupby('family_status_id')['debt'].count()
debt_family_status['debt_family_status_sum'] = credit.groupby('family_status_id')['debt'].sum()
debt_family_status['debt_family_status_ratio'] = (debt_family_status['debt_family_status_sum']/debt_family_status['debt_family_status_count'])
debt_family_status = debt_family_status.sort_values(by='debt_family_status_ratio', ascending=False)

debt_family_status_all = debt_family_status.merge (family, on = 'family_status_id', how = 'left')

debt_family_status_all

,family_status_id,debt_family_status_count,debt_family_status_sum,debt_family_status_ratio,index,family_status
0,4,2810,274,0.097509,24,Не женат / не замужем
1,1,4151,388,0.093471,4,гражданский брак
2,0,12339,931,0.075452,0,женат / замужем
3,3,1195,85,0.071130,19,в разводе
4,2,959,63,0.065693,18,вдовец / вдова


##### Вывод 2:

Одинокие заемщики (в разводе - 7.1%, вдовец / вдова - 6.6%) или заемщики в оформленных отношениях (женат / замужем - 7.5%) имеют просрочку реже, чем не Не женат / не замужем - 9.7% или гражданский брак - 9.3%

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [42]:
debt_income = pd.DataFrame()
debt_income['total_income_category_count'] = credit.groupby('total_income_category')['debt'].count()
debt_income['total_income_category_sum'] = credit.groupby('total_income_category')['debt'].sum()
debt_income['total_income_category_ratio'] = debt_income['total_income_category_sum']/debt_income['total_income_category_count']*100
debt_income = debt_income.sort_values(by='total_income_category_ratio', ascending=False)

debt_income

,total_income_category_count,total_income_category_sum,total_income_category_ratio
total_income_category,,,
E,22,2,9.090909
C,16015,1360,8.492039
A,25,2,8.000000
B,5042,356,7.060690
D,350,21,6.000000


#####  Вывод 3:

Люди со средним и выше среднего уровнем дохода чаще оказываются в должниках, чем с низким или высоким.

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

In [43]:
debt_purposes= pd.DataFrame()
debt_purposes['debt_purposes_count'] = credit.groupby('purpose_category')['debt'].count()
debt_purposes['debt_purposes_sum'] = credit.groupby('purpose_category')['debt'].sum()
debt_purposes['debt_purposes_ratio'] = debt_purposes['debt_purposes_sum']/debt_purposes['debt_purposes_count']*100
debt_purposes = debt_purposes.sort_values(by='debt_purposes_ratio', ascending=False)

debt_purposes

credit_pivot = credit.pivot_table(index=['purpose_category'],values='debt',aggfunc=['sum','count','mean'])
                                  
credit_pivot




,sum,count,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,403,4306,0.093590
операции с недвижимостью,782,10811,0.072334
получение образования,370,4013,0.092200
проведение свадеб,186,2324,0.080034


##### Вывод 4:

Из таблицы мы видим зависимость процента невозврата кредита в зависимости от целей его получения. Кредиты на покупку авто и образование имеют максимальный процент невозврата - 9,3% и 9,2% соотвественно. Операции с недвижимостью и проведение свадеб наименьшее - 7,2% и 8% соответсвенно.

## Общий вывод:

Была рассмотрена зависимость невозврата кредита от наличия детей у заемщика, уровня дохода и цели, на которую данный кредит брался.

Обнаружена зависимость невозврата кредита в срок от наличия детей. В целом уровень просроченных кредитов у заемщиком с детьми выше, чем без детей. 

Наибольший процент невозврата в срок у заемщиков с 4 детьми, чуть меньше с одним или двумя детьми. У заемщиков с 3 детьми самый низкий процент не возврата в срок. А вот у заемщиков с 5 детьми задолженность вообще отсутствует.

Заемщики, которые в разводе или вдовцы и заемщики, в оформленных отношениях, имеют просрочку реже, чем те кто не женат или не замужем, либо находятся в гражданском браке. Получается статус официальных отношений положительно сказывается на возврате кредита.

Люди со средним и выше среднего уровнем дохода чаще оказываются в должниках, чем с низким или высоким.

Кредиты на покупку авто и образование имеют максимальный процент невозврата кредита в срок. Операции с  недвижимостью наименьший.